In [1]:
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, models, layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50V2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

ImportError: cannot import name 'comb'

In [ ]:
INIT_LR = 1e-05  #1e-05  # This value is specific to what model is chosen: Inception, VGG or ResnNet.
EPOCHS = 30 
BS = 8

print("Loading images...")
imagePath = "./all/train"
imagePaths = list(paths.list_images(imagePath))

data = []
labels = []

# read all X-Rays in the specified path, and resize them all to 256x256
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    data.append(image)
    labels.append(label)
    
#normalise pixel values to real numbers between 0.0 - 1.0 
data = np.array(data) / 255.0
labels = np.array(labels)

# perform one-hot encoding for a 3-class labeling 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(integer_encoded)

print("... ... ", len(data), "images loaded in 3x classes:")
print(label_encoder.classes_)

In [ ]:
(x_train, x_val, y_train, y_val) = train_test_split(data, labels, test_size=0.20, stratify=labels)

In [ ]:
model = Sequential()
adam_s = Adam(learning_rate = 0.00001)

#model.add(VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='average'))
model.add(ResNet50V2(input_shape=(256, 256, 3),include_top=False, weights='imagenet',pooling='average'))

for layer in model.layers:
    layer.trainable = False

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam_s, metrics=['accuracy'])


model.summary()

In [ ]:
# train the head of the network
print("Training the full stack model...")
H = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=8)

In [ ]:
import tensorflow
model.save('resnet50.h5')

In [ ]:
new_model = tensorflow.keras.models.load_model('resnet50.h5')

In [ ]:
new_model.save('new-covid/1')

In [ ]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',	# Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0' 	# If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

new_model.save("s3://model/new-covid/1")

In [ ]:
print("[INFO] loading untrained test images...")
imagePathTest = "./all/test"
imagePathsTest = list(paths.list_images(imagePathTest))
print(len(imagePathsTest))

dataTest = []
labelsTest = []

# loop over the image paths
for imagePathTest in imagePathsTest:

    # extract the class label from the filename
	  labelTest = imagePathTest.split(os.path.sep)[-2]
  
	  # load the image, swap color channels, and resize it to be a fixed
	  # 256x256 pixels while ignoring aspect ratio
	  imageTest = cv2.imread(imagePathTest)
	  imageTest = cv2.cvtColor(imageTest, cv2.COLOR_BGR2RGB)
	  imageTest = cv2.resize(imageTest, (256, 256))

	  # update the data and labels lists, respectively
	  dataTest.append(imageTest)
	  labelsTest.append(labelTest)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 255]
dataTest = np.array(dataTest) / 255.0
labelsTest = np.array(labelsTest)


# perform one-hot encoding on the labels
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labelsTest)
labelsTest = to_categorical(integer_encoded)

print(labelsTest)

In [ ]:
predTest = new_model.predict(dataTest, batch_size=BS)
print(predTest)

In [ ]:
predClasses = predTest.argmax(axis=-1)
print(predClasses)

In [ ]:
!curl http://35.206.247.237:32380/v1/models/covid19

In [ ]:
!pip install tqdm
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import json
import requests
from tqdm import tqdm




#get some sample data
sample_test_data = dataTest  #[0:2]
sample_test_labels = labelsTest #[0:2]

print(sample_test_data.shape)

class_names = ['covid', 'normal', 'pneumonia_bac']

# pre-process data 
#sample_test_data_processed = np.expand_dims(sample_test_data / 255., axis=3)

sample_test_data_processed = sample_test_data

# create payload
data = json.dumps({"signature_name": "serving_default", 
                   "instances": sample_test_data_processed.tolist()})

In [ ]:
with open("image_data.json", "w") as json_file:
    json.dump(data, json_file)
# print(data)

In [ ]:
!curl -v -H "Host: covid19.kubeflow.example.com" -X POST http://35.206.247.237:32380/v1/models/covid19:predict -d@image_data.json

In [ ]:
HEADERS = {'content-type': 'application/json'}
#MODEL1_API_URL = 'http://172.17.0.1:8511/v1/models/covid19/versions/2:predict'
MODEL1_API_URL = 'http://35.206.247.237:32380/v1/models/covid19:predict'

# inference request
json_response = requests.post(MODEL1_API_URL, data=data, headers=HEADERS)

print(json_response)

pred = json.loads(json_response.content.decode('utf-8'))
print(pred)

# view server response
predictions = json.loads(json_response.text)['predictions']
predictions = np.argmax(np.array(predictions), axis=1)
prediction_labels = [class_names[p] for p in predictions]

fig, ax = plt.subplots(6, 5, figsize=(60, 80))
for idx, img in enumerate(sample_test_data):
    rowidx = idx // 5
    colidx = idx % 5
    ax[rowidx, colidx].imshow(img)
    ax[rowidx, colidx].set_title('Actual: {}\nPredicted: {}'.format(class_names[sample_test_labels[idx].argmax(axis=-1)], 
                                                                    prediction_labels[idx]), fontsize=40)